#Instructions

This document is a template to help you get started and will mirror the work that you will do in modules 2, 3, 4 and 5 with the Taxi Trip dataset problem.

You should save a copy of this in your Colab and change the name of the file to include your student number.

Within this document there are comments to help you along and some boilerplate code that you can adjust to get you started but the code will be very similar to that found in the practice document.

This document has the following sections and should be submitted with those in place:



*   Title
*   Introduction
*   Module 2: Get the data
*   Module 3: Basic statistics and visualisations
*   Module 4: Regression models
*   Module 5: Using the outcomes 


Enjoy and learn lots.

# Problem: Can we accurately predict the number of collisions for any given day of the week?

##Introduction

You work as a product owner for a car insurance company offering a daily insurance policy for car rentals.   

The company operates in New York and wants to price its insurance to reflect collision risk and associated costs. It wants you to explore a new feature for development that will make better predictions about this. We will use New York traffic collision data to make estimates about the number of collisions on a given day.  

For this you require weather data as there has been a link between weather and traffic collisions. The company is using data given to them by the emergency services.

Note: You will be given a file entitled collisions_and_weather_data.csv, testdata2019.csv. Due to Covid-19, all data since early 2020 has been fairly useless with respect to patterns. The company can see that the data has recently returned to full pre-pandemic levels and you will be provided data from 1st of January 2013 to 31st of December 2018 and the test data will be from 2019.   

Remember, you will have to put these files in your Google Drive.

MY THOUGHTS ON THE ASSUMPTIONS:

- We don't have data on number of journeys. Collisions is likely related to number of journeys, but we can't normalise w.r.t. this, or even test it as a hypothesis, since we don't have that data.

## Module 2: Get the data

This section contains boilerplate code. As long as you have uploaded your CSV files to your Google Drive, you can just run the cells as normal.

In [164]:
import string

# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as pl
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
import tensorflow as tf

from scipy import stats
from IPython.display import display
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error




In [165]:
#set the size of our plots as they are a little small by default.
plt.rcParams["figure.figsize"] = (20,5)

In [166]:
# Custom functions and constants
custom_plasma_scale = [
    [0, px.colors.sequential.Plasma[0]],  # Start of the Plasma scale
    # Map points in between as needed; this example directly jumps to clamping
    [0.8, px.colors.sequential.Plasma[-2]],  # Roughly the 80% mark in the Plasma palette
    [1, px.colors.sequential.Plasma[-2]]  # Clamp the color scale to the color at 80%
]
OICD = 'OICD_grouped_by_week_day'
OICD_YEAR_DAY = 'OICD_grouped_by_year_day'
MED_RES = 'median_residual'
N_COLS = 'NUM_COLLISIONS'
PERIODIC_RESIDUAL = 'periodic_residual'


# I have standardized a year to 366 days, so in order for day_of_year to match for both leap years and non-leap years, March onwards need to be adjusted +1 in the non-leap years.
def adjust_for_leap_year(year, month, day_of_year):
    adjustment = 0 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 1
    return day_of_year + adjustment if month > 2 else day_of_year
    

# This function converts every day of the year to a value in the range 1-2562, i.e. a Cartesian product of the days of the week and days of the year, with all the Sundays first, then Saturdays, etc..
def convert_date_to_one_indexed_cycle_day_grouped_by_day_of_week(year, month, day, day_num):
    
    temp_day_num = day_num 
    date = dt.datetime(year, month, day)
    day_of_year = date.timetuple().tm_yday

    adjusted_day_of_year = adjust_for_leap_year(year, month, day_of_year) # correct so March 1st is always 61 of the year. 

    day_of_total_cycle = adjusted_day_of_year + (temp_day_num * 366)
    return day_of_total_cycle


# This function does the same, but groups by year instead of day of the week.
def convert_date_to_one_indexed_cycle_day_grouped_by_day_of_year(year, month, day, day_num):
    
    temp_day_num = day_num 
    date = dt.datetime(year, month, day)
    day_of_year = date.timetuple().tm_yday

    adjusted_day_of_year = adjust_for_leap_year(year, month, day_of_year) # correct so March 1st is always 61 of the year. 

    # Calculate day of total cycle with the adjusted day_of_year
    day_of_total_cycle = (adjusted_day_of_year + temp_day_num) + (6 * adjusted_day_of_year)

    return day_of_total_cycle



def show_correlation_matrix(df, title):
    corrMatrix = df.corr(numeric_only=True)
    fig = px.imshow(
        corrMatrix,
        width=1000,
        height=1000,
        title=title,
        color_continuous_scale='PiYG',
        zmin=-1,
        zmax=1
        )
    fig.update_yaxes(tickfont=dict(family='Century Gothic', size=14), range=[-1,1])
    fig.update_xaxes(tickfont=dict(family='Century Gothic', size=14))

    alt = "Hello"
    fig.show()


days_titles = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat' ]

# So that I can make music later on. I saved the FFT as a sample and made some ambient EDM.
def output_wav(yf):
    # Create a wave
    ifft_result = ifft(yf)
    
    # Make real first
    real_ifft_result = np.real(ifft_result)
    
    # Normalise to -1 to 1
    normalized_ifft = np.interp(real_ifft_result, (real_ifft_result.min(), real_ifft_result.max()), (-1, +1))
    
    # Ensure it's real...
    audio_signal = np.real(normalized_ifft)
    
    # Rate
    sampling_rate = 96_000 # 96kHz for Serum.
    
    # Write to WAV
    write("collisions_of_NYC.wav", sampling_rate, audio_signal.astype(np.float32))

In [167]:
# Link with your google drive
# from google.colab import drive
# drive.mount('/content/gdrive')

In [168]:
# get our collated taxi trip and weather data from google drive (or in my case my local storage) TODO Swap for the Google Cloud link before submitting.
df = pd.read_csv('D:\\Coding\\JupyterNotebookLBD\\scientificProject\\data\\LBD_New_York_collisions_and_weather_data (1).csv')
# Replace null data values with 
df.replace({9999.9: np.nan, 999.9: np.nan, 99.99: np.nan}, inplace=True)

original_df = df.copy()

print('Initial dataframe, with null data instead of max values')
display(df.head())
display(df.describe())

mean_by_day = df.groupby('day')[N_COLS].mean()
print("Mean by day:")
print(mean_by_day)

# Dropping records missing 'mxpsd' as that shows a weak correlation but has missing data in some rows. (Count: 102 rows dropped)
df = df.dropna(axis=0, subset=['mxpsd']) 
display(df.describe())

# Re-order the day_nums so the correlation is stronger
df['day'] = (df['day'] + 1) % 7 # % 7 already moves Sunday to zero-indexed 0, so adding 1 also moves Saturday.
df.loc[df['day'] < 2, 'day'] = 1 - df['day'] # Swap Saturday and Sunday because the linear experience of time is arbitrary.

# Group by 'year' and calculate the median of 'NUM_COLLISIONS', then subtract it. This allows us to effectively eliminate the correlation to year, controlling for the unknown number of total journeys which is likely to be the causative externality.
df[MED_RES] = df[N_COLS] - df.groupby('year')[N_COLS].transform('median')


df[OICD] = df.apply(lambda row: convert_date_to_one_indexed_cycle_day_grouped_by_day_of_week(row['year'], row['mo'], row['da'], row['day']), axis=1)

df[OICD_YEAR_DAY] = df.apply(lambda row: convert_date_to_one_indexed_cycle_day_grouped_by_day_of_year(row['year'], row['mo'], row['da'], row['day']), axis=1)

print('Dataframe after adding OICD columns:')
display(df.describe())
df.head()


Initial dataframe, with null data instead of max values


,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS
0,2,2013,1,1,01/01/2013,37.8,23.6,1011.9,10.0,6.1,8.9,19.0,39.9,33.1,0.0,NaN,0,381
1,3,2013,1,2,02/01/2013,27.1,10.5,1016.8,10.0,5.3,9.9,19.0,33.1,21.9,0.0,NaN,0,480
2,4,2013,1,3,03/01/2013,28.4,14.1,1020.6,10.0,3.7,8.0,15.0,32.0,24.1,0.0,NaN,0,549
3,5,2013,1,4,04/01/2013,33.4,18.6,1017.0,10.0,6.5,13.0,24.1,37.0,30.0,0.0,NaN,0,505
4,6,2013,1,5,05/01/2013,36.1,18.7,1020.6,10.0,6.6,12.0,21.0,42.1,32.0,0.0,NaN,0,389


,day,year,mo,da,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS
count,2191.000000,2191.000000,2191.000000,2191.000000,2191.000000,2191.000000,2168.000000,2159.000000,2089.000000,2089.000000,1376.000000,2191.000000,2191.000000,2191.000000,176.000000,2191.000000,2191.000000
mean,4.000000,2015.500228,6.523962,15.726609,55.721086,41.120310,1017.225046,8.953682,4.533605,9.311776,20.205523,65.226974,47.875947,0.141031,6.427273,0.079416,602.121862
std,2.000457,1.707859,3.449207,8.800821,17.506851,19.298085,7.205239,1.563377,2.050030,3.114821,4.706593,18.156330,17.152164,0.353569,4.723467,0.270448,102.452173
min,1.000000,2013.000000,1.000000,1.000000,6.900000,-16.100000,992.100000,1.700000,0.000000,2.900000,14.000000,17.600000,-0.900000,0.000000,1.200000,0.000000,10.000000
25%,2.000000,2014.000000,4.000000,8.000000,41.550000,26.400000,1012.600000,8.450000,3.100000,7.000000,17.100000,50.000000,35.100000,0.000000,2.000000,0.000000,533.000000
50%,4.000000,2016.000000,7.000000,16.000000,56.900000,42.600000,1017.000000,9.800000,4.300000,8.900000,19.000000,66.900000,48.000000,0.000000,5.900000,0.000000,604.000000
75%,6.000000,2017.000000,10.000000,23.000000,71.900000,57.500000,1021.725000,10.000000,5.700000,11.100000,22.900000,82.000000,64.000000,0.080000,9.100000,0.000000,670.000000
max,7.000000,2018.000000,12.000000,31.000000,89.100000,74.800000,1042.100000,10.000000,15.500000,24.100000,40.000000,98.100000,82.900000,4.530000,22.000000,1.000000,1161.000000


Mean by day:
day
1    603.185304
2    626.808307
3    621.057508
4    635.878594
5    673.884984
6    559.916933
7    494.121406
Name: NUM_COLLISIONS, dtype: float64


,day,year,mo,da,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS
count,2089.000000,2089.000000,2089.000000,2089.000000,2089.000000,2089.000000,2084.000000,2077.000000,2089.000000,2089.000000,1376.000000,2089.000000,2089.000000,2089.000000,172.000000,2089.000000,2089.000000
mean,3.994256,2015.404978,6.364289,15.630445,56.214265,41.482001,1017.179367,8.954357,4.533605,9.311776,20.205523,65.824031,48.315223,0.139694,6.437209,0.082336,600.501675
std,1.999513,1.669737,3.382665,8.829828,17.557594,19.393204,7.149609,1.559558,2.050030,3.114821,4.706593,18.156722,17.229664,0.354351,4.749319,0.274942,99.814277
min,1.000000,2013.000000,1.000000,1.000000,6.900000,-16.100000,992.100000,1.700000,0.000000,2.900000,14.000000,18.000000,-0.900000,0.000000,1.200000,0.000000,188.000000
25%,2.000000,2014.000000,3.000000,8.000000,42.100000,26.700000,1012.600000,8.400000,3.100000,7.000000,17.100000,51.100000,35.100000,0.000000,2.000000,0.000000,533.000000
50%,4.000000,2015.000000,6.000000,16.000000,58.000000,43.400000,1016.900000,9.800000,4.300000,8.900000,19.000000,68.000000,50.000000,0.000000,5.900000,0.000000,603.000000
75%,6.000000,2017.000000,9.000000,23.000000,72.300000,57.900000,1021.700000,10.000000,5.700000,11.100000,22.900000,82.000000,64.000000,0.080000,9.100000,0.000000,667.000000
max,7.000000,2018.000000,12.000000,31.000000,89.100000,74.800000,1042.100000,10.000000,15.500000,24.100000,40.000000,98.100000,82.900000,4.530000,22.000000,1.000000,999.000000


Dataframe after adding OICD columns:


,day,year,mo,da,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_week_day,OICD_grouped_by_year_day
count,2089.000000,2089.000000,2089.000000,2089.000000,2089.000000,2089.000000,2084.000000,2077.000000,2089.000000,2089.000000,1376.000000,2089.000000,2089.000000,2089.000000,172.000000,2089.000000,2089.000000,2089.000000,2089.000000,2089.000000
mean,3.000957,2015.404978,6.364289,15.630445,56.214265,41.482001,1017.179367,8.954357,4.533605,9.311776,20.205523,65.824031,48.315223,0.139694,6.437209,0.082336,600.501675,-3.164672,1277.159885,1254.667305
std,1.997124,1.669737,3.382665,8.829828,17.557594,19.393204,7.149609,1.559558,2.050030,3.114821,4.706593,18.156722,17.229664,0.354351,4.749319,0.274942,99.814277,94.913364,738.354970,724.910997
min,0.000000,2013.000000,1.000000,1.000000,6.900000,-16.100000,992.100000,1.700000,0.000000,2.900000,14.000000,18.000000,-0.900000,0.000000,1.200000,0.000000,188.000000,-415.000000,1.000000,7.000000
25%,1.000000,2014.000000,3.000000,8.000000,42.100000,26.700000,1012.600000,8.400000,3.100000,7.000000,17.100000,51.100000,35.100000,0.000000,2.000000,0.000000,533.000000,-64.000000,634.000000,632.000000
50%,3.000000,2015.000000,6.000000,16.000000,58.000000,43.400000,1016.900000,9.800000,4.300000,8.900000,19.000000,68.000000,50.000000,0.000000,5.900000,0.000000,603.000000,0.000000,1277.000000,1248.000000
75%,5.000000,2017.000000,9.000000,23.000000,72.300000,57.900000,1021.700000,10.000000,5.700000,11.100000,22.900000,82.000000,64.000000,0.080000,9.100000,0.000000,667.000000,61.000000,1918.000000,1864.000000
max,6.000000,2018.000000,12.000000,31.000000,89.100000,74.800000,1042.100000,10.000000,15.500000,24.100000,40.000000,98.100000,82.900000,4.530000,22.000000,1.000000,999.000000,392.000000,2561.000000,2567.000000


,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,...,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_week_day,OICD_grouped_by_year_day
0,3,2013,1,1,01/01/2013,37.8,23.6,1011.9,10.0,6.1,...,19.0,39.9,33.1,0.0,NaN,0,381,-177.0,1099,10
1,4,2013,1,2,02/01/2013,27.1,10.5,1016.8,10.0,5.3,...,19.0,33.1,21.9,0.0,NaN,0,480,-78.0,1466,18
2,5,2013,1,3,03/01/2013,28.4,14.1,1020.6,10.0,3.7,...,15.0,32.0,24.1,0.0,NaN,0,549,-9.0,1833,26
3,6,2013,1,4,04/01/2013,33.4,18.6,1017.0,10.0,6.5,...,24.1,37.0,30.0,0.0,NaN,0,505,-53.0,2200,34
4,1,2013,1,5,05/01/2013,36.1,18.7,1020.6,10.0,6.6,...,21.0,42.1,32.0,0.0,NaN,0,389,-169.0,371,36


## Module 3: Basic statistics and visualisations

In [169]:
# df = df.sort_values(["year", "mo", "da"], ascending = (True, True, True)) # order the data by year, month, day in ascending order.
df = df.sort_values([OICD], ascending=(True)) # order by OICD
display(df.head()) # check the data again by viewing the first 5 rows


,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,...,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_week_day,OICD_grouped_by_year_day
1461,0,2017,1,1,01/01/2017,44.8,22.9,1017.9,10.0,5.0,...,21.0,48.0,43.0,0.00,NaN,0,485,-151.0,1,7
1097,0,2016,1,3,03/01/2016,38.4,20.0,1011.7,10.0,6.4,...,21.0,45.0,32.0,0.00,NaN,0,432,-203.0,3,21
733,0,2015,1,4,04/01/2015,45.4,42.3,1010.6,5.8,5.1,...,19.0,55.9,33.1,0.89,NaN,0,381,-222.0,4,28
369,0,2014,1,5,05/01/2014,30.3,21.9,1025.6,6.4,3.8,...,15.0,36.0,27.0,0.00,5.9,1,320,-248.0,5,35
5,0,2013,1,6,06/01/2013,38.3,25.0,1019.5,8.5,5.3,...,17.1,46.0,34.0,0.00,NaN,0,393,-165.0,6,42


In [170]:

# Initial Correlation Overview
show_correlation_matrix(df, "Collision Correlations")

def scatter_x_y(x, y): px.scatter(
    df,
    x=x,
    y=y,
    color='day',
    color_continuous_scale=custom_plasma_scale,  # Use the custom Plasma scale
    title=y + ' / '+ x).show()
    


scatter_x_y(OICD, N_COLS)
scatter_x_y(OICD, MED_RES)
scatter_x_y(OICD_YEAR_DAY, N_COLS)
scatter_x_y(OICD_YEAR_DAY, MED_RES)




As expected, by using the residuals against the year-median collisions, we have strengthened the correlation w.r.t. the periodic behaviour. The last two scatter plots show the clear decrease in linear correlation comparing the raw collisions for the OICD grouped by year, and the same ordering but against the median residuals.

We will now remove the outliers, in order to find a harmonic regression that isn't disturbed by aperiodic black swans.

In [171]:


# Get ZICD and collisions
x = df[OICD]
y = df[MED_RES]

# Do linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

# Calculate Residuals
predicted_y = slope * x + intercept
residuals = y - predicted_y

# Determine Outlier threshold
residual_std = np.std(residuals)
mean_residual = np.mean(residuals)

# We are going to use an threshold of 2 standard deviations, which we can expect to remove the 5% furthest from the mean, assuming the samples conform to an approximate normal distribution. It is generally a reasonable assumption that car collisions are independent probability events - the exception being very large outlier accidents, but these are the unpredictable events we wish to remove anyway.
SIGMA_LIMIT = 2

outlier_upper = mean_residual + SIGMA_LIMIT * residual_std
outlier_lower = mean_residual - SIGMA_LIMIT * residual_std


cleaned_df = df[(residuals <= outlier_upper) & (residuals >= outlier_lower)]

print('Total before cleaning:')
display(df.count())
print('Total after cleaning:')
display(cleaned_df.count())
print('Our original data set for comparison:')
display(original_df.count())
print('Number of rows culled: ', len(original_df) - len(cleaned_df))



linear_reg_cleaned_fig = px.scatter(
    cleaned_df,
    x=OICD,
    y=MED_RES,
    color='day',
    color_continuous_scale=custom_plasma_scale,  # Use the custom Plasma scale
    title=MED_RES + ' / ' + OICD + ' - Cleaned via ' + str(SIGMA_LIMIT) + 'sigma' + ' linear regression'
)
    
linear_reg_cleaned_fig.show()

show_correlation_matrix(cleaned_df, "Collision Outliers removed")


# Assuming `df` is your initial DataFrame
# Shuffle the DataFrame rows
df_shuffled = cleaned_df.sample(frac=1, random_state=42).reset_index(drop=True)


# Calculate the number of rows to cull
num_to_cull = int(len(df) * 0.2)

# Split the data
test_df = df_shuffled[:num_to_cull]  # This will be your test set (20% of the data)
train_df = df_shuffled[num_to_cull:]  # This will be your training set (80% of the data)
display(df_shuffled.head())
display(train_df.head())



Total before cleaning:


day                         2089
year                        2089
mo                          2089
da                          2089
collision_date              2089
temp                        2089
dewp                        2089
slp                         2084
visib                       2077
wdsp                        2089
mxpsd                       2089
gust                        1376
max                         2089
min                         2089
prcp                        2089
sndp                         172
fog                         2089
NUM_COLLISIONS              2089
median_residual             2089
OICD_grouped_by_week_day    2089
OICD_grouped_by_year_day    2089
dtype: int64

Total after cleaning:


day                         1987
year                        1987
mo                          1987
da                          1987
collision_date              1987
temp                        1987
dewp                        1987
slp                         1982
visib                       1975
wdsp                        1987
mxpsd                       1987
gust                        1295
max                         1987
min                         1987
prcp                        1987
sndp                         160
fog                         1987
NUM_COLLISIONS              1987
median_residual             1987
OICD_grouped_by_week_day    1987
OICD_grouped_by_year_day    1987
dtype: int64

Our original data set for comparison:


day               2191
year              2191
mo                2191
da                2191
collision_date    2191
temp              2191
dewp              2191
slp               2168
visib             2159
wdsp              2089
mxpsd             2089
gust              1376
max               2191
min               2191
prcp              2191
sndp               176
fog               2191
NUM_COLLISIONS    2191
dtype: int64

Number of rows culled:  204


,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,...,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_week_day,OICD_grouped_by_year_day
0,0,2015,9,13,13/09/2015,72.0,62.5,1005.9,8.6,5.2,...,22.0,80.1,66.9,0.29,NaN,0,478,-125.0,257,1799
1,5,2018,4,12,12/04/2018,51.5,33.2,1016.8,10.0,3.8,...,20.0,63.0,37.0,0.00,NaN,0,610,-17.0,1933,726
2,2,2014,9,15,15/09/2014,62.8,43.4,1022.2,10.0,2.5,...,NaN,71.1,53.1,0.00,NaN,0,610,42.0,991,1815
3,4,2014,2,5,05/02/2014,31.8,27.5,1015.2,4.1,6.4,...,24.1,35.1,21.9,0.97,7.9,1,468,-100.0,1500,256
4,6,2014,2,28,28/02/2014,16.5,-8.0,1024.3,10.0,6.1,...,24.1,34.0,9.0,0.00,5.9,0,655,87.0,2255,419


,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,...,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_week_day,OICD_grouped_by_year_day
417,1,2013,6,15,15/06/2013,69.4,52.0,1013.6,10.0,3.5,...,NaN,80.1,61.0,0.06,NaN,0,596,38.0,533,1170
418,1,2018,7,28,28/07/2018,75.6,69.0,1015.1,6.5,0.5,...,NaN,86.0,68.0,0.25,NaN,0,637,10.0,576,1471
419,3,2016,4,12,12/04/2016,54.0,40.6,1016.3,9.1,5.3,...,22.0,64.9,43.0,0.04,NaN,0,638,3.0,1201,724
420,3,2015,2,10,10/02/2015,30.0,19.3,1016.7,9.6,8.1,...,20.0,39.9,25.0,0.07,7.9,0,571,-32.0,1139,290
421,4,2018,4,4,04/04/2018,46.7,39.0,1004.2,3.9,6.0,...,35.0,55.9,39.0,0.26,NaN,1,554,-73.0,1559,669


It is clear from looking at the scatter plots of both OICD and OICD_grouped_by_year that alongside the weekly cycle of collision numbers, there are other periodic functions. Some obvious human factors to consider are: academic or public holidays. How busy are people in their daily lives and are they sticking to routine or making extra, unusual journeys? It doesn't matter specifically what these are: it matters that people deviate from their regular patterns and this is usually accompanied by increased risk. The weather aspect also is periodic - we normally call these "seasons"! If the weather has a effect on the number of collisions, this will show a predictable periodic aspect, as the weather is not itself random overall.

To investigate these phenomena, we will take a FFT of the cleaned data we have so far, and extract any significant signals from it.  

In [172]:


from scipy.fft import fft, ifft, fftfreq
from scipy.interpolate import interp1d
from scipy.io.wavfile import write

train_df = train_df.sort_values(by=OICD)

display(train_df.head())

clean_x = train_df[OICD]
clean_y = train_df[MED_RES]

# Since this is now an obvious linear correlation w.r.t. day of the week (OICD), we will find this and removed it from the signal
# Do linear regression
clean_slope, clean_intercept, clean_r_value, clean_p_value, clean_std_err = stats.linregress(clean_x,clean_y)

# Calculate Residuals from this regression.
clean_predicted_y = clean_slope * clean_x + clean_intercept
clean_residuals = clean_y - clean_predicted_y


,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,...,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_week_day,OICD_grouped_by_year_day
1170,0,2017,1,1,01/01/2017,44.8,22.9,1017.9,10.0,5.0,...,21.0,48.0,43.0,0.00,NaN,0,485,-151.0,1,7
1722,0,2016,1,3,03/01/2016,38.4,20.0,1011.7,10.0,6.4,...,21.0,45.0,32.0,0.00,NaN,0,432,-203.0,3,21
442,0,2015,1,4,04/01/2015,45.4,42.3,1010.6,5.8,5.1,...,19.0,55.9,33.1,0.89,NaN,0,381,-222.0,4,28
995,0,2014,1,5,05/01/2014,30.3,21.9,1025.6,6.4,3.8,...,15.0,36.0,27.0,0.00,5.9,1,320,-248.0,5,35
1488,0,2013,1,6,06/01/2013,38.3,25.0,1019.5,8.5,5.3,...,17.1,46.0,34.0,0.00,NaN,0,393,-165.0,6,42


We now have two separate compenents: the simple linear regression w.r.t. OICD, and some noisy-looking periodic components.

In [173]:

px.line(x=clean_x, y=clean_predicted_y, title="Linear regression line").show()
px.scatter(x=clean_x, y=clean_residuals, title="Cleaned residuals w.r.t. linear regression").show()

regular_x = np.arange(clean_x.min(),clean_x.max()+1,1)

f = interp1d(clean_x, clean_residuals, kind='linear')
regular_y = f(regular_x)

regular_line = interp1d(clean_x, clean_predicted_y, kind='linear')
regular_y_line = f(regular_x)

px.line(x=regular_x, y=regular_y_line, title="Periodic function regression").show()

# Plot the interpolated samples
px.line(df, x=regular_x, y=regular_y, title="Interpolated samples").show()

From the periodic signal data, we will take the FFT and plot the signal amplitude against frequency.

In [174]:

# Number of samples
N = len(regular_y)

# Spacing interpolated to be 1/366
T = 1 / 2562


yf = fft(regular_y)
xf = fftfreq(N, T)[0:N//2]
wavelength_in_days = 1/fftfreq(N, T)[1:N//2]

# magnitude = 2.0/N * np.abs(yf[1:N//2])
magnitude_with_first_value = 2.0/N * np.abs(yf[0:N//2])
# transform_plot = px.line(x=wavelength_in_days,y=magnitude, title="FFT Magnitude For Wavelength")
# transform_plot.update_layout(xaxis_title="Wavelength in days", yaxis_title="Magnitude")
# transform_plot.show()

px.line(x=xf, y = magnitude_with_first_value, title="FFT Magnitude for Frequency").show()



<h1>How do we interpret this plot?</h1>
<ol>
<li>It is indeed noisy, but there are some obvious spikes at the low frequency end</li>
<li>The length of our defined cycle is 2562, so dividing this number by any frequency gives us the wavelength in days that signal
<li>These signals are noteworthily significant: 
<ul><li>2562 / 7 = 366, i.e. a year*.</li><li>2562 / 14 = i.e. 6 months</li> <li>2562 / 56 = every 45 days, or 6 weeks. Could be related to academic semesters.</li> <li>2562 / 105 = every 24 days, or 3 weeks.</li>   
</ol>

<p> These are off-the-cuff human, or seasonal, phenomena that I'm suggesting, to give a concrete basis for whether these signals are plausible. Again: it isn't necessary to answer the <strong>why</strong> at this stage, we are looking for the <strong>what</>.</p>

<p> We will now clean this signal using standard time-domain noise threshold approaches, in order to extract a simplified,  <em>smoothed out</em>, model to reinforce the linear regression we have bookmarked.

*(Remember that we are treating every year as if it were a leap year, because people don't plan according to a date being "the 61st day of the year", they plan for it to be March 1st) 

In [175]:
cut_off_frequency = 1200
cut_off_index = int(cut_off_frequency * N / T)

yf[cut_off_index:-cut_off_index] = 0

#  yf is the FFT of the data and xf is the frequency bins
magnitude = np.abs(yf)


# These three operations are defined as functions, so that later I can loop through them incrementing the parameters to find the optimum noise threshold for extracting a signal that fits the data.

# Top level function
def find_r2(threshold_factor):
    predicted_regular_y, prediction_df = create_prediction_df(threshold_factor)

    linear_prediction_df = pd.DataFrame({
        OICD: regular_x,
        'Prediction': predicted_regular_y
    })
    
    linear_mse, r2_linear, mae_linear = compare_pred_to_df(linear_prediction_df)

    mse, r2, mae = compare_pred_to_df(prediction_df)

    return np.array([threshold_factor, mse, r2, mae, linear_mse, r2_linear, mae_linear])

# Function to compare the model to the real data
def compare_pred_to_df(prediction_df):
    # Merge true values with modulated line
    test_predictions = pd.merge(test_df[[OICD, MED_RES]], prediction_df, on=OICD, how='left')
    # Drop rows where 'NUM_COLLISIONS' or 'Prediction' is NaN
    test_predictions_clean = test_predictions.dropna(subset=[MED_RES, 'Prediction'])
    y_true = test_predictions_clean[MED_RES]
    y_pred = test_predictions_clean['Prediction']
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return mse, r2, mae

# Function to extract the model using a given noise threshold.
def create_prediction_df(threshold_factor):

    # Identify significant frequencies based on a threshold
    threshold = magnitude.max() * threshold_factor  # Example threshold: 10% of max amplitude8
    significant_indices = np.where(magnitude >= threshold)[0]

    # Create a filtered version of the FFT results
    filtered_yf = np.zeros_like(yf)
    filtered_yf[significant_indices] = yf[significant_indices]

    # Inverse FFT to get the significant cyclical signal
    significant_signal = np.real(ifft(filtered_yf))
    predicted_regular_y = clean_slope * regular_x + clean_intercept
 
    # Combine with linear regression predictions
    combined_prediction = predicted_regular_y + significant_signal

    prediction_df = pd.DataFrame({
        OICD: regular_x,
        'Prediction': combined_prediction
    })
    return predicted_regular_y, prediction_df


outputs = []
# From prior experimentation I've picking a starting value of 0.19 for the noise threshold, as the outcome drops below this.
# For clarification: we're dropping from our FFT any signal that has a magnitude less than (strongest * threshold) 
baseline = 0.19

for factor in range(1000):
    outputs.append(find_r2(baseline+factor/10000))
    
r2_values = np.array(outputs)[:,2]
mae_values = np.array(outputs)[:,3]
threshold_used = np.array(outputs)[:,0]
linear_r2 = np.array(outputs)[:,5]
linear_mae = np.array(outputs)[:,6]

# We put all the results in a dataframe, so we can pick a winner.
df_best_threshold = pd.DataFrame({
    'Threshold Used': threshold_used,
    'R2': r2_values,
    'MAE': mae_values,
    'Linear_R2': linear_r2,
    'Linear_MAE': linear_mae
})

fig = go.Figure()


fig.add_trace(go.Scatter(x=df_best_threshold['Threshold Used'], y=df_best_threshold['R2'], mode='lines', name='R2 vs Threshold'))
fig.add_trace(go.Scatter(x=df_best_threshold['Threshold Used'], y=df_best_threshold['Linear_R2'], mode='lines', name='Linear_R2 vs Threshold'))

fig_mae = go.Figure()
fig_mae.add_trace(go.Scatter(x=df_best_threshold['Threshold Used'], y=df_best_threshold['MAE'], mode='lines', name='MAE vs Threshold'))
fig_mae.add_trace(go.Scatter(x=df_best_threshold['Threshold Used'], y=df_best_threshold['Linear_MAE'], mode='lines', name='Linear_MAE vs Threshold'))
fig.show()
fig_mae.show()


<html >
  <head>


    
  </head>
  <body>
    

<p>
My original outcome here was:
</p>
<ul>
<li>Mean Squared Error (MSE): 4170.4539197977165</li>
<li>R-squared: 0.5029024872423475</li>
</ul>
<p>

This was without correcting for leap years in the OICD calculation (i.e. I wasn't ensuring that March 1st was always day 61 of the year), and without removing the general year-on-year increase by using year_median residuals (the MED_RES column).

So, this is a small but meaningful increase in the predictive strength by making these changes.

I also found by trial and error* that 2sigma was the degree of outlier cleaning to get the strongest harmonic regression. Beyond that and black swans stole my lunch; below that, and we lost some outliers that seem to have a periodic quality to them.

Finally, the above plot is the R-squared value of the range of signal thresholds shown on the x-axis. The optimal threshold for the "noise-gate":
<ul>
<li>0.238, which gives an R-squared value of 0.61 and 
<li>MAE of 39.7</li> 
</ul>

We will now merge this periodic signal with the simple linear regression that we removed beforehand, and input that into the dataframe so we can look at its correlation strength with the actual data. We will also find the residuals w.r.t. this model, so we can see what (if any) other data points might be valuable to explore the remaining 40% of feature variance.

*I did consider creating a meta loop to generate the cartesian product of sigma-outlier values and noise-gate thresholds, but on balance that seems to fall into over-engineering/over-fitting.
</p>

  </body>
</html>


In [176]:
predicted_regular_y, df_best_threshold  = create_prediction_df(0.238)

# Merge the periodic prediction (modulated linear regression, I will later refer to as MLR) with the dataframe that still has all 2191 rows.
df_with_periodic_prediction = pd.merge(df_best_threshold[['Prediction', OICD]], df, on=OICD, how='left')
df_with_periodic_prediction.dropna(axis=0, how='any', subset='temp', inplace=True)
df_with_periodic_prediction[PERIODIC_RESIDUAL] = df_with_periodic_prediction['median_residual'] - df_with_periodic_prediction['Prediction']


show_correlation_matrix(df_with_periodic_prediction, title='Now with periodic-based residuals')

If I've understood this correctly, then there is <em>no significant additional predictive power</em> gained by considering the weather. That is not to say the weather doesn't influence collisions: it may do, but weather is inherently seasonal, as are many aspects of human behaviour. By controlling for the established periodic cycle of the number of collisions (across the 2562 OICD range), we find no further significant correlation with the weather data. Put another way, if rain is forecast on a specific day in December, that is less important than the general prediction we can already make from the given day in December. We know it rains in December The same is true of the other weather factors.

Three more observations can be made: 
1. The correlation between the periodic-based predication, and the number of collisions is the strongest in the matrix (0.65), suggesting the additional work in of doing the FFT was justified.
2. The periodic residual is moderately correlated with the NUM_COLLISIONS (0.67), which implies the logical scenario that the more collisions there are on a given day, the more likely it is that this is an unusual day.
3. The predicted value has itself a correlation with the weather factors, similar to the original dataset.
 
This last point is especially important: it implies that the weather is not directly a strong cause of collisions. The collisions change according to the time of year (for a number of reasons), and the weather also changes according to the time of the year. Although it is tempting to make mechanistic hypotheses like "wet roads make braking harder", the data doesn't support the assertion that even a perfect weather forecast would be of significant value in predicting the number of collisions. Since we can't even have a perfect weather forecast, we are absolutely justified in ignoring it.





## Module 4: Regression models

We will now explore whether the machine learning models can improve on the MLR, by considering some of the details we excluded.

In [177]:

from tensorflow import keras
from tensorflow.keras import layers

# We will use the median residual again, so that the comparison is valid.
one_input_data = [df[OICD], df[MED_RES]] # array of only the OICD and collisions
headers = [OICD, MED_RES] # Titles of input and output
df_one_input = pd.concat(one_input_data, axis=1, keys=headers) # Make the DF from the data and headers
df_one_input.head()



,OICD_grouped_by_week_day,median_residual
1461,1,-151.0
1097,3,-203.0
733,4,-222.0
369,5,-248.0
5,6,-165.0


In [178]:
# 0.8 Split, as in the MLR work flow.
def train_test_split(df_input):
    train_data_output = df_input.sample(frac=0.8, random_state=0)
    test_data_output = df_input.drop(train_data_output.index)
    return train_data_output, test_data_output

In [179]:

train_data, test_data = train_test_split(df_one_input)

train_data.describe()

,OICD_grouped_by_week_day,median_residual
count,1671.000000,1671.000000
mean,1275.937762,-2.929384
std,740.814387,94.797343
min,1.000000,-415.000000
25%,622.500000,-65.000000
50%,1283.000000,0.000000
75%,1920.500000,62.000000
max,2561.000000,392.000000


In [180]:
# Using a scale factor of 500 as the median residuals are in the range roughly -500 to 500, so we can multiply by 1000 to easily interpret the outputs. 
scale_factor_labels = 500

def get_features_and_labels(train_data_i, test_data_i):
    train_features_o = train_data_i.copy()
    test_features_o = test_data_i.copy()
    
    train_labels_o = train_features_o.pop(MED_RES)
    test_labels_o = test_features_o.pop(MED_RES)
    
    train_labels_o = train_labels_o / scale_factor_labels
    test_labels_o = test_labels_o / scale_factor_labels
    
    return train_features_o.astype(float), test_features_o.astype(float), train_labels_o.astype(float), test_labels_o.astype(float)

train_features, test_features, train_labels, test_labels = get_features_and_labels(train_data, test_data)

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
    print('First example:', first)
    print()
    print('Normalized: ', normalizer(first).numpy()) 

First example: [[278.]]

Normalized:  [[-1.35]]


In [181]:
one_indexed_cycle_day_inputs = np.array(train_features[OICD])
oicd_normalizer = layers.Normalization(input_shape=[1, ], axis=None)
oicd_normalizer.adapt(one_indexed_cycle_day_inputs)

oicd_model = tf.keras.Sequential([
    oicd_normalizer,
    layers.Dense(units=1)
])
oicd_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_7 (Normaliza  (None, 1)                 3         
 tion)                                                           
                                                                 
 dense_13 (Dense)            (None, 1)                 2         
                                                                 
Total params: 5 (24.00 Byte)
Trainable params: 2 (8.00 Byte)
Non-trainable params: 3 (16.00 Byte)
_________________________________________________________________


In [182]:
oicd_model.predict(one_indexed_cycle_day_inputs[:10])

1/1 [==============================] - 0s 39ms/step


array([[ 1.614863  ],
       [ 0.7264712 ],
       [-1.2218415 ],
       [-1.3156972 ],
       [ 0.48374125],
       [-0.29137653],
       [-1.4742806 ],
       [ 1.02422   ],
       [ 0.6617432 ],
       [ 1.5727898 ]], dtype=float32)

In [183]:
oicd_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error'
)

In [184]:
%%time
history = oicd_model.fit(
    train_features[OICD],
    train_labels,
    epochs=100,
    verbose=0,
    validation_split=0.2
)

CPU times: total: 7.83 s
Wall time: 5.98 s


In [185]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,val_loss,epoch
95,0.126685,0.126969,95
96,0.125891,0.140178,96
97,0.128167,0.124537,97
98,0.124797,0.119410,98
99,0.127028,0.131419,99


In [186]:
def plot_loss(history, title):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history['epoch'], y=history['loss'], mode='lines', name='loss vs epoch'))
    fig.add_trace(go.Scatter(x=history['epoch'], y=history['val_loss'], mode='lines', name='val_loss vs epoch'))
    fig.update_layout(title=title)
    fig.show()

In [187]:
    
plot_loss(hist, "Linear Model")

In [188]:

test_results = {}

test_results['OICD_model'] = oicd_model.evaluate(
    test_features[OICD],
    test_labels, verbose=0
) * scale_factor_labels


def plot_ZICD(x_input, y_input):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_input, mode='markers', y=y_input,  name='Predictions'))
    fig.add_trace(go.Scatter(x=train_features[OICD], mode='markers', y=train_labels, name='Data'))
    fig.show()

x = tf.linspace(0,2561, 2562)
y = oicd_model.predict(x)


    
x_flat = x.numpy()
y_flat = y.flatten()

plot_ZICD(x_flat, y_flat)
    

81/81 [==============================] - 0s 613us/step


This is a good first outcome: a plausible straight line through the scatter of OICD values. Using the same function defined in module 3, we can compare the predictions of this model with the full data set that we started with.

In [189]:
y_scaled = y_flat*scale_factor_labels

ml_pred_df = pd.DataFrame({
    OICD: x_flat,
    'Prediction': y_scaled
})

mean_squared, r_squared, mean_absolute = compare_pred_to_df(ml_pred_df)
print("Mean squared error: ", mean_squared)
print("R-squared: ", r_squared)
print("Mean absolute error: ", mean_absolute)

Mean squared error:  5349.728077316528
R-squared:  0.2656059756220095
Mean absolute error:  59.96898831109063


<h1>DNN Model</h1>

As you would expect from the simplicity of the linear model, it does not have as strong predictive power as the final MLR model.

We will attempt to build a more nuanced prediction using a DNN model. Two versions of the data will be used: one that is still restricted only to the temporal values (i.e. OICD, but using one-hot encoding), and another using the weather data also. 

In [220]:
def one_hot_days(df):
    df_one_hot = df.copy()
    df_one_hot['day'] = df_one_hot['day'].map({0: 'Zero', 1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', 6: 'Six'})
    df_one_hot = pd.get_dummies(df_one_hot, columns=['day'], prefix='', prefix_sep='')
    return df_one_hot
df_one_hot = one_hot_days(df)

In [221]:
def one_hot_months_and_years(df_one_hot_input):
    df_one_hot_input['mo'] = df_one_hot_input['mo'].map({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'})
    return  pd.get_dummies(df_one_hot_input, columns=['mo'], prefix='', prefix_sep='')
df_one_hot = one_hot_months_and_years(df_one_hot)

In [243]:
def get_dnn_many_inputs(df_one_hot_input):
     return [df_one_hot_input['One'], df_one_hot_input['Two'], df_one_hot_input['Three'], df_one_hot_input['Four'], df_one_hot_input['Five'], df_one_hot_input['Six'], df_one_hot_input['Zero'], df_one_hot_input['Jan'], df_one_hot_input['Feb'], df_one_hot_input['Mar'], df_one_hot_input['Apr'], df_one_hot_input['May'], df_one_hot_input['Jun'], df_one_hot_input['Jul'], df_one_hot_input['Aug'], df_one_hot_input['Sep'], df_one_hot_input['Oct'], df_one_hot_input['Nov'], df_one_hot_input['Dec'], df_one_hot_input[MED_RES]]

dnn_many_input_data = get_dnn_many_inputs(df_one_hot)
def get_dnn_many_inputs_with_weather(df_one_hot_input):
    return [df_one_hot_input['One'], df_one_hot_input['Two'], df_one_hot_input['Three'], df_one_hot_input['Four'], df_one_hot_input['Five'], df_one_hot_input['Six'], df_one_hot_input['Zero'], df_one_hot_input['Jan'], df_one_hot_input['Feb'], df_one_hot_input['Mar'], df_one_hot_input['Apr'], df_one_hot_input['May'], df_one_hot_input['Jun'], df_one_hot_input['Jul'], df_one_hot_input['Aug'], df_one_hot_input['Sep'], df_one_hot_input['Oct'], df_one_hot_input['Nov'], df_one_hot_input['Dec'], df_one_hot_input[MED_RES], df_one_hot_input['temp'], df_one_hot_input['dewp'], df_one_hot_input['slp'], df_one_hot_input['visib'], df_one_hot_input['wdsp'], df_one_hot_input['mxpsd'], df_one_hot_input['max'], df_one_hot_input['min'], df_one_hot_input['prcp'], df_one_hot_input['fog']]

dnn_with_weather =get_dnn_many_inputs_with_weather(df_one_hot) 
headers = ['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', MED_RES]
headers_with_weather = ['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', MED_RES, 'temp', 'dewp', 'slp', 'visib', 'wdsp', 'mxpsd', 'max', 'min', 'prcp', 'fog']
df_dnn_many_input_data = pd.concat(dnn_many_input_data, axis=1, keys=headers)

df_dnn_weather= pd.concat(dnn_with_weather, axis=1, keys=headers_with_weather)

df_dnn_weather.dropna(axis=1, inplace=True)
df_dnn_weather.head()   

,Zero,One,Two,Three,Four,Five,Six,Jan,Feb,Mar,...,Dec,median_residual,temp,dewp,wdsp,mxpsd,max,min,prcp,fog
1461,False,False,False,False,False,False,True,True,False,False,...,False,-151.0,44.8,22.9,5.0,9.9,48.0,43.0,0.00,0
1097,False,False,False,False,False,False,True,True,False,False,...,False,-203.0,38.4,20.0,6.4,9.9,45.0,32.0,0.00,0
733,False,False,False,False,False,False,True,True,False,False,...,False,-222.0,45.4,42.3,5.1,8.9,55.9,33.1,0.89,0
369,False,False,False,False,False,False,True,True,False,False,...,False,-248.0,30.3,21.9,3.8,7.0,36.0,27.0,0.00,1
5,False,False,False,False,False,False,True,True,False,False,...,False,-165.0,38.3,25.0,5.3,8.9,46.0,34.0,0.00,0


We now have two parallel dataframes that we will use to create comparative DNN models to add to the MLR and linear ML models.

In [193]:
# Many input no weather
train_data_many_input, test_data_many_input = train_test_split(df_dnn_many_input_data)
train_features_many_input, test_features_many_input, train_labels_many_input, test_labels_many_input  = get_features_and_labels(train_data_many_input, test_data_many_input)

# Including the weather
train_data_we, test_data_we = train_test_split(df_dnn_weather)
train_features_we, test_features_we, train_labels_we, test_labels_we  = get_features_and_labels(train_data_we, test_data_we)

print(train_features_we)

def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1),
    ])
    
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features_many_input))

first = np.array(train_features_many_input[:1])

with np.printoptions(precision=2, suppress=True):
    print('First example:', first)
    print()
    print('Normalized:', normalizer(first).numpy())
    
     
normalizer_we = tf.keras.layers.Normalization(axis=-1)
print(train_features_we.isna().sum())
print(np.isinf(train_features_we).sum())

sample_data = np.random.rand(100,5)

normalizer_we.adapt(np.array(train_features_we)) 

dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

dnn_model_weather = build_and_compile_model(normalizer_we)
dnn_model_weather.summary()

      Zero  One  Two  Three  Four  Five  Six  Jan  Feb  Mar  ...  Nov  Dec  \
1006   0.0  0.0  0.0    0.0   0.0   0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0   
1189   0.0  1.0  0.0    0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2025   0.0  0.0  0.0    0.0   1.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1353   0.0  0.0  0.0    0.0   1.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
608    0.0  1.0  0.0    0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...    ...  ...  ...    ...   ...   ...  ...  ...  ...  ...  ...  ...  ...   
1531   0.0  0.0  0.0    0.0   0.0   0.0  1.0  0.0  0.0  1.0  ...  0.0  0.0   
994    0.0  0.0  1.0    0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
983    0.0  0.0  0.0    0.0   0.0   1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
581    0.0  0.0  1.0    0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1486   0.0  0.0  0.0    0.0   1.0   0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0   

      temp  dewp  wdsp  mxpsd   max   min  prcp  fog  
1006  56

In [194]:
%%time
history_dnn = dnn_model.fit(
    train_features_many_input,
    train_labels_many_input,
    epochs=100,
    verbose=0,
    validation_split=0.2
)

CPU times: total: 9.64 s
Wall time: 6.83 s


In [195]:
%%time
history_dnn_weather = dnn_model_weather.fit(
    train_features_we,
    train_labels_we,
    epochs=100,
    verbose=0,
    validation_split=0.2
)

CPU times: total: 8.83 s
Wall time: 6.46 s


In [196]:
hist_dnn = pd.DataFrame(history_dnn.history)
hist_dnn['epoch'] = history_dnn.epoch
plot_loss(hist_dnn, "Weather Agnostic DNN Training")

In [197]:
hist_dnn_we = pd.DataFrame(history_dnn_weather.history)
hist_dnn_we['epoch'] = history_dnn_weather.epoch
plot_loss(hist_dnn_we, "Weather Aware DNN Training")

The "Weather Aware" Model seems to suffer from over-fitting, as the difference between the loss and validated loss grows steadily as the training goes on. 

In [198]:
test_results['dnn_model'] = dnn_model.evaluate(test_features_many_input, test_labels_many_input, verbose=0) * scale_factor_labels
pd.DataFrame(test_results, index=['Mean absolute error '+MED_RES]).T

,Mean absolute error median_residual
OICD_model,65.833040
dnn_model,54.125011


In [199]:
test_results['dnn_weather'] = dnn_model_weather.evaluate(test_features_we, test_labels_we, verbose=0) * scale_factor_labels
pd.DataFrame(test_results, index=['Mean absolute error ' + MED_RES]).T

,Mean absolute error median_residual
OICD_model,65.833040
dnn_model,54.125011
dnn_weather,62.930517


As was suggested by the overfitting from the loss graph: the weather-aware DNN performs worse than the temporal-only DNN model, though it is (insignificantly) better than the OICD linear model. However, even the mean absolute error of the temporal only DNN, at 46.8, is not an improvement on the MLR model, which gave a mean absolute error of 39.7.

My recommendation moving forward will be to use the MLR (modulated linear regression model), for two reasons:
<ol>
    <li>It is measurably <strong>more accurate</strong>.</li>
    <li>It is <strong>intuitive to explain</strong> to the non-data science stakeholders. It represents a numerical version of what we understand instinctively: humans have different behavioural patterns at different times of the year.
</ol>

<h1>Module 5: Using the outcomes</h1>

In this section you want to use the test data to test what kind of money you will potentially make. 

Your company rents cars daily to people in New York City and is struggling in a saturated market. You have noted that you offer a flat rate damage waiver insurance package to all customers and that most customers chose not to take it. This package is something that has the potential to make the company lots of money if marketed properly.

At the moment you offer the package for a fee of 30 dollars per day, with only around 30% of all customers taking it. You rent on average 20,000 vehicles per day and therefore this package makes the company 180,000 dollars. The damage caused by collisions costs on average 500 dollars per collision with 8% of customers encountering a collision of some kind resulting in damage. The total costs from damage come to 800,000 dollars, which is covered by the customers' insurance, but around 10% of this is covered by the company due to fradulent behaviour or customers taking the waiver. This results in a profit of around 100,000 dollars per day for the sale of this package alone. 

This 30 dollars is based on an expected 1,200 collisions per day (based on the maximum).

The goal of this investigation is to accurately predict the number of expected collisions on a given day in order to reduce the price of the on-demand package and therefore give value to the customer. Surveys have shown that a competitive price would result in 80% of respondents taking the damage waiver insurance option – but the price must reflect the associated costs.

In [279]:
def accountancy(number):
    return "$" + format(number, ",.2f")

# only 30% current uptake
current_uptake  = 0.3

# rent on average 20_000 vehicles per day
daily_rented_vehicles = 20_000

# Current waiver price
current_waiver_fee = 30

# Total income from waiver package
income_from_package_per_day = current_uptake * daily_rented_vehicles * current_waiver_fee

print("Current average waiver package income: ", accountancy(income_from_package_per_day))

Current average waiver package income:  $180,000.00


In [280]:
# A collision costs on average $500
cost_of_collision = 500

# 10% of collision cost covered by the company
covered_by_company = 0.1

# Current SOP is to assume a ceiling of 1_200 collisions per day
max_num_collisions = 1_200

# Assuming 80% of customers could be induced to take the waiver, what is the lowest fee that wouldn't result in a loss of earnings from the package as it stands:
potential_uptake = 0.8
potential_min_fee = income_from_package_per_day/daily_rented_vehicles/potential_uptake
print("Potential minimum fee: $", potential_min_fee)

Potential minimum fee: $ 11.25


In [281]:
# Income from the package over 90 days:
income_90_days = income_from_package_per_day * 90
print("Income over 90 days from the waiver package @ $180,000 / day: ", accountancy(income_90_days))

Income over 90 days from the waiver package @ $180,000 / day:  $16,200,000.00


In [282]:
# Cost Ceiling 90 days @ 1,200 collisions per day:
expected_costs_90_days = cost_of_collision * covered_by_company * max_num_collisions * 90
print("Worst case scenario-style accounting: ", accountancy(expected_costs_90_days))

Worst case scenario-style accounting:  $5,400,000.00


In [283]:
# Projected profit:
profit_90_days = income_90_days - expected_costs_90_days
print("Pessimistic profit projection: ", accountancy(profit_90_days))

Pessimistic profit projection:  $10,800,000.00


<h3>The Story So Far</h3>
This is the current SOP for the company. We hope to improve on this projection using our chosen model (MLR), so that we can lower the figure for the expected costs and free-up liquidity in the company's financial assets.   


In [284]:
df_2019_test_data = pd.read_csv('D:\Coding\JupyterNotebookLBD\scientificProject\data\LBD_testdata2019.csv')

In [285]:
df_2019_test_data = df_2019_test_data.sort_values(["year", "mo", "da"], ascending = (True, True, True))

In [286]:
df_2019_test_data.head()

,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS
0,2,2019,1,1,01/01/2019,50.5,43.2,1009.8,7.0,999.9,999.9,999.9,57.9,36.0,1.08,999.9,0,430
1,3,2019,1,2,02/01/2019,38.0,23.2,1024.2,10.0,999.9,999.9,999.9,57.9,35.1,0.06,999.9,0,502
2,4,2019,1,3,03/01/2019,41.1,29.4,1015.8,9.9,999.9,999.9,999.9,44.1,35.1,0.00,999.9,0,504
3,5,2019,1,4,04/01/2019,39.7,26.4,1014.8,9.9,999.9,999.9,999.9,46.0,35.1,0.00,999.9,0,598
4,6,2019,1,5,05/01/2019,44.2,41.0,1003.3,5.3,999.9,999.9,999.9,46.9,35.1,0.22,999.9,0,455


The first transformation we need to make in order to compare the real 2019 data with our MLR predictions is to apply the same OICD encoding. We can then merge the real data with the output of the MLR model.

In [287]:
df_2019_test_data.replace({9999.9: np.nan, 999.9: np.nan, 99.99: np.nan}, inplace=True)

# Re-order the day_nums so the correlation is stronger
df_2019_test_data['day'] = (df_2019_test_data['day'] + 1) % 7 # % 7 already moves Sunday to zero-indexed 0, so adding 1 also moves Saturday.
df_2019_test_data.loc[df_2019_test_data['day'] < 2, 'day'] = 1 - df_2019_test_data['day'] # Swap Saturday and Sunday because the linear experience of time is arbitrary.

# Group by 'year' and calculate the median of 'NUM_COLLISIONS', then subtract it. This allows us to effectively eliminate the correlation to year, controlling for the unknown number of total journeys which is likely to be the causative externality.
df_2019_test_data[MED_RES] = df_2019_test_data[N_COLS] - df_2019_test_data.groupby('year')[N_COLS].transform('median')


df_2019_test_data[OICD] = df_2019_test_data.apply(lambda row: convert_date_to_one_indexed_cycle_day_grouped_by_day_of_week(row['year'], row['mo'], row['da'], row['day']), axis=1)

df_2019_test_data[OICD_YEAR_DAY] = df_2019_test_data.apply(lambda row: convert_date_to_one_indexed_cycle_day_grouped_by_day_of_year(row['year'], row['mo'], row['da'], row['day']), axis=1)

print('Dataframe after adding OICD columns:')
display(df_2019_test_data.describe())
df_2019_test_data.head()

Dataframe after adding OICD columns:


,day,year,mo,da,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_week_day,OICD_grouped_by_year_day
count,365.0,365.0,365.000000,365.000000,365.000000,365.000000,364.000000,365.000000,295.000000,295.000000,164.000000,365.000000,365.000000,365.000000,11.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,3.0,2019.0,6.526027,15.720548,55.495890,41.613151,1017.064011,9.063288,4.026780,8.747797,19.675610,64.959726,47.179726,0.153479,1.763636,0.095890,578.224658,-10.775342,1281.838356,1289.868493
std,2.0,0.0,3.452584,8.808321,17.096162,19.192213,7.473048,1.522007,1.882138,2.935981,4.728871,17.839615,16.816924,0.326679,1.082841,0.294845,93.538521,93.538521,738.533432,740.204964
min,0.0,2019.0,1.000000,1.000000,8.400000,-13.000000,992.600000,2.400000,0.700000,4.100000,14.000000,15.100000,1.900000,0.000000,1.200000,0.000000,10.000000,-579.000000,6.000000,10.000000
25%,1.0,2019.0,4.000000,8.000000,40.600000,27.000000,1011.975000,8.700000,2.700000,7.000000,15.900000,50.000000,33.100000,0.000000,1.200000,0.000000,522.000000,-67.000000,645.000000,654.000000
50%,3.0,2019.0,7.000000,16.000000,56.600000,44.500000,1017.150000,9.900000,3.600000,8.000000,18.100000,66.900000,48.900000,0.000000,1.200000,0.000000,589.000000,0.000000,1282.000000,1291.000000
75%,5.0,2019.0,10.000000,23.000000,70.800000,57.700000,1022.300000,10.000000,5.050000,9.900000,22.000000,81.000000,62.100000,0.120000,1.600000,0.000000,641.000000,52.000000,1918.000000,1928.000000
max,6.0,2019.0,12.000000,31.000000,88.700000,73.600000,1037.000000,10.000000,10.700000,20.000000,34.000000,95.000000,82.000000,2.190000,3.900000,1.000000,803.000000,214.000000,2558.000000,2565.000000


,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,...,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_week_day,OICD_grouped_by_year_day
0,3,2019,1,1,01/01/2019,50.5,43.2,1009.8,7.0,NaN,...,NaN,57.9,36.0,1.08,NaN,0,430,-159.0,1099,10
1,4,2019,1,2,02/01/2019,38.0,23.2,1024.2,10.0,NaN,...,NaN,57.9,35.1,0.06,NaN,0,502,-87.0,1466,18
2,5,2019,1,3,03/01/2019,41.1,29.4,1015.8,9.9,NaN,...,NaN,44.1,35.1,0.00,NaN,0,504,-85.0,1833,26
3,6,2019,1,4,04/01/2019,39.7,26.4,1014.8,9.9,NaN,...,NaN,46.0,35.1,0.00,NaN,0,598,9.0,2200,34
4,1,2019,1,5,05/01/2019,44.2,41.0,1003.3,5.3,NaN,...,NaN,46.9,35.1,0.22,NaN,0,455,-134.0,371,36


In [288]:
df_best_threshold.describe()

,OICD_grouped_by_week_day,Prediction
count,2561.000000,2561.000000
mean,1281.000000,0.009596
std,739.441343,65.232350
min,1.000000,-201.659036
25%,641.000000,-35.631519
50%,1281.000000,3.461625
75%,1921.000000,45.511712
max,2561.000000,150.077258


In [289]:
# Merge the periodic prediction (MLR), the DNNs and linear model with the dataframe for 2019 .
df_2019_with_periodic_prediction = pd.merge(df_best_threshold[['Prediction', OICD]], df_2019_test_data, on=OICD, how='left')

df_2019_with_periodic_prediction[PERIODIC_RESIDUAL] = df_2019_with_periodic_prediction['median_residual'].sub(df_2019_with_periodic_prediction['Prediction'], fill_value=None)

fig_2019_data = go.Figure()

fig_2019_data.add_trace(go.Scatter(x=df_2019_with_periodic_prediction[OICD], y=df_2019_with_periodic_prediction[MED_RES], mode='markers', name='Real Data 2019'))
fig_2019_data.add_trace(go.Scatter(x=df_2019_with_periodic_prediction[OICD], y=df_2019_with_periodic_prediction['Prediction'], mode='lines', name='MLR Prediction'))
fig_2019_data.update_layout(title='Real 2019 Data against MLR Prediction')

fig_2019_data.show()
display(df_2019_with_periodic_prediction.describe())

,Prediction,OICD_grouped_by_week_day,day,year,mo,da,temp,dewp,slp,visib,...,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,median_residual,OICD_grouped_by_year_day,periodic_residual
count,2561.000000,2561.000000,365.0,365.0,365.000000,365.000000,365.000000,365.000000,364.000000,365.000000,...,164.000000,365.000000,365.000000,365.000000,11.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,0.009596,1281.000000,3.0,2019.0,6.526027,15.720548,55.495890,41.613151,1017.064011,9.063288,...,19.675610,64.959726,47.179726,0.153479,1.763636,0.095890,578.224658,-10.775342,1289.868493,-11.015452
std,65.232350,739.441343,2.0,0.0,3.452584,8.808321,17.096162,19.192213,7.473048,1.522007,...,4.728871,17.839615,16.816924,0.326679,1.082841,0.294845,93.538521,93.538521,740.204964,72.751813
min,-201.659036,1.000000,0.0,2019.0,1.000000,1.000000,8.400000,-13.000000,992.600000,2.400000,...,14.000000,15.100000,1.900000,0.000000,1.200000,0.000000,10.000000,-579.000000,10.000000,-546.321976
25%,-35.631519,641.000000,1.0,2019.0,4.000000,8.000000,40.600000,27.000000,1011.975000,8.700000,...,15.900000,50.000000,33.100000,0.000000,1.200000,0.000000,522.000000,-67.000000,654.000000,-47.924291
50%,3.461625,1281.000000,3.0,2019.0,7.000000,16.000000,56.600000,44.500000,1017.150000,9.900000,...,18.100000,66.900000,48.900000,0.000000,1.200000,0.000000,589.000000,0.000000,1291.000000,-3.187031
75%,45.511712,1921.000000,5.0,2019.0,10.000000,23.000000,70.800000,57.700000,1022.300000,10.000000,...,22.000000,81.000000,62.100000,0.120000,1.600000,0.000000,641.000000,52.000000,1928.000000,32.369891
max,150.077258,2561.000000,6.0,2019.0,12.000000,31.000000,88.700000,73.600000,1037.000000,10.000000,...,34.000000,95.000000,82.000000,2.190000,3.900000,1.000000,803.000000,214.000000,2565.000000,162.050859


In [290]:
fig_prediction_residual = go.Figure()

fig_prediction_residual.add_trace(go.Scatter(x=df_2019_with_periodic_prediction[OICD], y=df_2019_with_periodic_prediction[
    PERIODIC_RESIDUAL], mode='markers', name=PERIODIC_RESIDUAL))
fig_prediction_residual.update_layout(title=PERIODIC_RESIDUAL)

<h3>Observations</h3>
The MLR model shows a strong fit against the real 2019 data. Most usefully though, it seems to over estimate more than it underestimates. The most egregious outliers are in the negative part of the graph (i.e. the true collision number was significantly lower than predicted), and the general spread is well-centred around 0.  

In [337]:
# We can now calculate what the outcome would have been were we to have used this model in 2019.
real_collision_total = df_2019_test_data.sort_values('day')[N_COLS][:90].sum()
actual_costs_first_90_days = (real_collision_total * cost_of_collision * covered_by_company)
difference_actual_pessimistic = actual_costs_first_90_days - expected_costs_90_days
print("Actual costs for first 90 days: ", accountancy(actual_costs_first_90_days))
print("Liquidity unnecessarily retained during this period: ", accountancy(difference_actual_pessimistic))

Actual costs for first 90 days:  $2,272,200.00
Liquidity unnecessarily retained during this period:  $-3,127,800.00


In [324]:
# Get the median for 2018 to use as our offset for "what we would have predicted in 2019"
df_median_by_year = df.groupby('year')[N_COLS].median()
print(df_median_by_year)
median_2018 = df_median_by_year.loc[2018]
print("2018: ", median_2018)

year
2013    558.0
2014    568.0
2015    603.0
2016    635.0
2017    636.0
2018    627.0
Name: NUM_COLLISIONS, dtype: float64
2018:  627.0


In [325]:

def generate_predictions(model, test_data, key):
    return model.predict(test_data[key][:90]) * scale_factor_labels + median_2018
    
linear_day_predictions = generate_predictions(oicd_model, df_2019_test_data, OICD)


3/3 [==============================] - 0s 2ms/step


In [326]:
one_hot_2019 = one_hot_days(df_2019_test_data)
one_hot_2019 = one_hot_months_and_years(one_hot_2019)
 
dnn_inputs_2019 = get_dnn_many_inputs(one_hot_2019)
dnn_weather_inputs_2019 = get_dnn_many_inputs_with_weather(one_hot_2019)

df_dnn_many_input_data_2019 = pd.concat(dnn_inputs_2019, axis=1, keys=headers)
df_dnn_many_input_data_2019.drop(MED_RES, axis=1, inplace=True)

df_dnn_many_input_data_2019.dropna(axis=1, inplace=True)

df_dnn_inputs_weather_2019= pd.concat(dnn_weather_inputs_2019, axis=1, keys=headers_with_weather)
df_dnn_inputs_weather_2019.dropna(axis=1, inplace=True)

df_dnn_many_input_data_2019 = df_dnn_many_input_data_2019.map(lambda x: 1 if x is True else 0 if x is False else x)
df_dnn_inputs_weather_2019 = df_dnn_inputs_weather_2019.map(lambda x: 1 if x is True else 0 if x is False else x)

display(df_dnn_inputs_weather_2019.head())
display(df_dnn_weather.head())



,Zero,One,Two,Three,Four,Five,Six,Jan,Feb,Mar,...,Nov,Dec,median_residual,temp,dewp,visib,max,min,prcp,fog
0,0,0,1,0,0,0,0,1,0,0,...,0,0,-159.0,50.5,43.2,7.0,57.9,36.0,1.08,0
1,0,0,0,1,0,0,0,1,0,0,...,0,0,-87.0,38.0,23.2,10.0,57.9,35.1,0.06,0
2,0,0,0,0,1,0,0,1,0,0,...,0,0,-85.0,41.1,29.4,9.9,44.1,35.1,0.00,0
3,0,0,0,0,0,1,0,1,0,0,...,0,0,9.0,39.7,26.4,9.9,46.0,35.1,0.00,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,-134.0,44.2,41.0,5.3,46.9,35.1,0.22,0


,Zero,One,Two,Three,Four,Five,Six,Jan,Feb,Mar,...,Dec,median_residual,temp,dewp,wdsp,mxpsd,max,min,prcp,fog
1461,False,False,False,False,False,False,True,True,False,False,...,False,-151.0,44.8,22.9,5.0,9.9,48.0,43.0,0.00,0
1097,False,False,False,False,False,False,True,True,False,False,...,False,-203.0,38.4,20.0,6.4,9.9,45.0,32.0,0.00,0
733,False,False,False,False,False,False,True,True,False,False,...,False,-222.0,45.4,42.3,5.1,8.9,55.9,33.1,0.89,0
369,False,False,False,False,False,False,True,True,False,False,...,False,-248.0,30.3,21.9,3.8,7.0,36.0,27.0,0.00,1
5,False,False,False,False,False,False,True,True,False,False,...,False,-165.0,38.3,25.0,5.3,8.9,46.0,34.0,0.00,0


In [327]:

dnn_model_predictions_2019 = dnn_model.predict(df_dnn_many_input_data_2019[:90]) * scale_factor_labels + median_2018
dnn_weather_model_predictions_2019 = dnn_model_weather.predict(df_dnn_inputs_weather_2019[:90]) * scale_factor_labels + median_2018

print("Linear Predictions:", linear_day_predictions)
print("DNN Predictions:", dnn_model_predictions_2019)
print("DNN with Weather Predictions:", dnn_weather_model_predictions_2019)

3/3 [==============================] - 0s 1ms/step
Linear Predictions: [[585.2801 ]
 [606.7759 ]
 [628.2716 ]
 [649.76733]
 [542.6401 ]
 [521.2615 ]
 [564.19434]
 [585.6901 ]
 [607.18585]
 [628.6816 ]
 [650.17737]
 [543.05005]
 [521.67145]
 [564.6044 ]
 [586.1001 ]
 [607.5959 ]
 [629.0916 ]
 [650.58734]
 [543.4601 ]
 [522.0814 ]
 [565.01434]
 [586.51013]
 [608.00586]
 [629.5016 ]
 [650.9973 ]
 [543.87006]
 [522.49146]
 [565.4244 ]
 [586.9201 ]
 [608.41583]
 [629.9116 ]
 [651.40735]
 [544.28   ]
 [522.9014 ]
 [565.83435]
 [587.33014]
 [608.82587]
 [630.3216 ]
 [651.8173 ]
 [544.69006]
 [523.31146]
 [566.2444 ]
 [587.7401 ]
 [609.23584]
 [630.7316 ]
 [652.22736]
 [545.10004]
 [523.72144]
 [566.65436]
 [588.1501 ]
 [609.6459 ]
 [631.1416 ]
 [652.6373 ]
 [545.5101 ]
 [524.1315 ]
 [567.0644 ]
 [588.5601 ]
 [610.05585]
 [631.5516 ]
 [653.1059 ]
 [545.97864]
 [524.60004]
 [567.53296]
 [589.0287 ]
 [610.5244 ]
 [632.0202 ]
 [653.5159 ]
 [546.3886 ]
 [525.01   ]
 [567.94293]
 [589.43866]
 [610.

In [328]:
 
# Create an x-axis index that matches the predictions' length
predictions_index = np.arange(len(dnn_model_predictions_2019))

fig_prediction_dnn = go.Figure()

fig_prediction_dnn.add_trace(go.Scatter(x=df_2019_test_data[:90].index, y=df_2019_test_data[:90][N_COLS], mode='markers', name='Real Data'))
fig_prediction_dnn.add_trace(go.Scatter(x=predictions_index, y=dnn_model_predictions_2019.flatten(), mode='markers', name='DNN Prediction'))
fig_prediction_dnn.update_layout(title='PredictionVia DNN Model')
fig_prediction_dnn.show()


# Create an x-axis index that matches the predictions' length
predictions_index_weather = np.arange(len(dnn_weather_model_predictions_2019))

fig_prediction_dnn = go.Figure()

fig_prediction_dnn.add_trace(go.Scatter(x=df_2019_test_data[:90].index, y=df_2019_test_data[:90][N_COLS], mode='markers', name='Real Data'))
fig_prediction_dnn.add_trace(go.Scatter(x=predictions_index_weather, y=dnn_weather_model_predictions_2019.flatten(), mode='markers', name='DNN with Weather Prediction'))
fig_prediction_dnn.update_layout(title='PredictionVia DNN Weather-Aware Model')
fig_prediction_dnn.show()

In [338]:
# Set up a dataframe for all the prediction aggregates:
aggregate_headers = ['Model', '90-Day-Prediction']
df_aggregate_collisions = pd.DataFrame({header: [] for header in aggregate_headers})

# Real Data:
real_row = pd.DataFrame({'Model': 'REAL DATA', '90-Day': real_collision_total}, index=[0])

# MLR Collisions:
sorted_to_year_day = df_2019_with_periodic_prediction.sort_values('day')
mlr_with_2018_median_offset = sorted_to_year_day['Prediction'] + median_2018
mlr_90_day_collisions_prediction = mlr_with_2018_median_offset[:90].sum()

mlr_row = pd.DataFrame({'Model': 'MLR', '90-Day': mlr_90_day_collisions_prediction}, index=[1])

# Linear Collisions:
linear_collision_prediction_sum = linear_day_predictions.sum()
linear_row = pd.DataFrame({'Model': 'Linear', '90-Day': linear_collision_prediction_sum}, index=[2])

# DNN Collisions:
dnn_sum = dnn_model_predictions_2019.flatten().sum()
dnn_row = pd.DataFrame({'Model': 'DNN', '90-Day': dnn_sum}, index=[3])

# DNN Weather Collisions:
dnn_weather_sum = dnn_weather_model_predictions_2019.flatten().sum()
dnn_weather_row = pd.DataFrame({'Model': 'DNN Weather', '90-Day': dnn_weather_sum}, index=[4])

frames = [real_row, mlr_row, linear_row, dnn_row, dnn_weather_row]
result = pd.concat(frames)

result.head()

,Model,90-Day
0,REAL DATA,45444.000000
1,MLR,49262.437737
2,Linear,52933.039062
3,DNN,52366.804688
4,DNN Weather,65803.476562


Reassuringly, they are all in the same ballpark.

In [344]:
result['Predicted Costs'] = (result['90-Day'] * cost_of_collision * covered_by_company)
result['Liquidity Difference'] = result['Predicted Costs'] - expected_costs_90_days
result['Liquidity Difference'] = result['Liquidity Difference'].apply(accountancy)
result['Predicted Costs'] = result['Predicted Costs'].apply(accountancy)

result.head()

,Model,90-Day,Predicted Costs,Liquidity Difference
0,REAL DATA,45444.000000,"$2,272,200.00","$-3,127,800.00"
1,MLR,49262.437737,"$2,463,121.89","$-2,936,878.11"
2,Linear,52933.039062,"$2,646,651.95","$-2,753,348.05"
3,DNN,52366.804688,"$2,618,340.23","$-2,781,659.77"
4,DNN Weather,65803.476562,"$3,290,173.83","$-2,109,826.17"


In [345]:
total_waivers_sold_over_90_day = daily_rented_vehicles * 0.8 * 90
mlr_liquidity_prediction_per_waiver = 2_900_000 / total_waivers_sold_over_90_day
print("Amount by which waiver could be further reduced: ", accountancy(mlr_liquidity_prediction_per_waiver))

Amount by which waiver could be further reduced:  $2.01


In [346]:
print("Floor price predicted by MLR:", accountancy(potential_min_fee - mlr_liquidity_prediction_per_waiver))

Floor price predicted by MLR: $9.24


<h1>Where does this leave us?</h1>
<h3>Summary</h3>
<p>We've looked at four different approaches to modelling the data:</p> 
<ol><li>Modulated Linear Regression </li>
<li> Linear Regression via machine learning</li>
<li> DNN using only one-indexed-cycle-day data</li>
<li> and DNN with weather data included</li>
</ol>
<p>The four models all agree that the current SOP of assuming 1,200 collisions per day is significantly more than is reasonably required. The four models all make predictions that exceed the true data more than they underestimate it, which leaves us with some margin for leaner financial management, even if only taking taking even figure at face value.</p>
<h3>Proposed Course of Immediate Action</h3>
<p> One obvious suggestion from this outcome is that we could further drop the price of the waiver, to induce more customers to take it. Taking the leanest (and also most accurate) prediction, from the MLR model, we could drop the price to &#36;9.24. This would leave us with a further buffer of approximately &#36;200,000. However, a more prudent recommendation would be <strong>&#36;9.99</strong>, as this also fits with conventional pricing models. There is a lot to be said for the familiar when selling to people. Oddly-specific numbers have a tendency to put people ill-at-ease, and the sub-conscious rounding-down of &#36;9.99 to &#36;9 is a common gambit.</p>
 <h3>Further Development</h3>
 <p>As suggested by several staff member in the recent meeting, there is considerable exploratory scope for linking this model with our existing data warehouses, and testing whether other factors could influence the prediction:</p>
 <ul>
 <li>Finer-grained location and time data</li>
 <li>Demographic-related data</li>
 <li>Car-specific data (e.g. are some of our rental vehicles more frequently involved in collisions than others?)</li>
 </ul>
   <p>However, it is also worth noting that customers are increasingly wary of price-gouging and injudicious fluctation. We should also be extremely cautious about the possiblity of bias and discrimination, both legally and in terms of optics. A small, predictable fluctuation in price, e.g. +/- &#36;2 across a regular timeframe, e.g. rush-hour versus off-peak might be seen as fairer to all customers than extreme differences. These are all questions best answered by surveys, marketing and pilot schemes.</p> 